In [1]:
import os 
import json
import pprint
%matplotlib inline
from serializer import*
import tweepy
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from pyvis.network import Network
import os

#Twitter API credentials
api_key = "ENFvcEFBU6prbQWZVMKYHhHCW"
api_secret = "OoLxYiucfTgnheUH1Bp7YJTsyIto9VcxLERBeXCg84NwjjgCAr"
access_token = "2572511777-adhDu1rnH5RYVPdfk51jPzLbOzpppivKmVYTtFH"
access_secret = "Oq1Q8CTYoDKxfyal7aY2x5V88QBqM2SAFac5e8LV6eeWY"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJkTJQEAAAAAaoRAjF4yb5OBca0iPYQZvs5vH5s%3D5AG5ZaEGM6pFtGL7cQohdtzimqd1aLcqHGbJPOSCXO4Fxnltwz"

In [2]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')
    
s = Serializer("data")

Authentication completed successfully!


In [ ]:
users = ["mizzaro"]

for user in users:
    print(f"Processing @{user}")

    user_followers = []

    for item in tweepy.Cursor(
            api.friends,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
    ).items():
        user_followers.append(item._json)

    print(f"Found {len(user_followers)} following for @{user}")
    s.serialize_json(f"{user}/{user}_following.json", user_followers)

Processing @mizzaro


Rate limit reached. Sleeping for: 892


In [ ]:
###############################OLD###################################
import os

directory = "data"
G = nx.DiGraph(membri=["Lorenzo Bellina" "Francesco Bombassei De Bona", "Andrea Cantarutti", "Gabriele Dominici"])
for dirname in os.listdir(directory):
    if not dirname.startswith(".") and not dirname == "all_users.json":
        print(f"Analizzo {dirname}")
        s = Serializer(f"data/{dirname}")
        profile = s.read_json(f"{dirname}_profile.json")
        G.add_node(profile["id"], **profile)
        print(f'Aggiungo il nodo {profile["name"]}')

        followers = s.read_json(f"{dirname}_followers.json")
        print(f'Aggiungo i {len(followers)} followers di {profile["name"]}')
        for item in followers:
            G.add_node(item["id"], **item)
            G.add_edge(item["id"], profile["id"])

        following = s.read_json(f"{dirname}_following.json")
        print(f'Aggiungo i {len(following)} following di {profile["name"]}')
        for item in following:
            G.add_node(item["id"], **item)
            G.add_edge(profile["id"], item["id"])

        print(f'Inserisco i random di {profile["name"]}')
        temp_dir = directory+"/"+dirname
        for file in os.listdir(temp_dir):
            if file.startswith("random"):
                id_user = int(file.split("_")[1])
                if file.endswith("following.json"):
                    following = s.read_json(f"{file}")
                    print(f"Inserisco i {len(following)} following del random di id {id_user}")
                    for item in following:
                        G.add_node(item["id"], **item)
                        G.add_edge(id_user, item["id"])
                elif file.endswith("follower.json"):
                    followers = s.read_json(f"{file}")
                    print(f"Inserisco i {len(followers)} followers del random di id {id_user}")
                    for item in followers:
                        G.add_node(item["id"], **item)
                        G.add_edge(item["id"], id_user)
        node_it = G.in_degree()
        for (node, in_degree) in node_it:
            G.nodes[node]["follower_finded"] = in_degree 
            
nx.write_gpickle(G, "graph/graph_networkx.pkl")

In [4]:
def get_friendship(sourceid, targetid, api):
    kind = ""

    friendship = api.show_friendship(source_id=sourceid, target_id=targetid)

    if not friendship[0].following and not friendship[0].followed_by:
        kind = "none"
    elif not friendship[0].following and friendship[0].followed_by:
        kind = "r_l"
    elif friendship[0].following and not friendship[0].followed_by:
        kind = "l_r"
    else:
        kind = "bi"

    return {
        "source_id": sourceid,
        "target_id": targetid,
        "friendship": kind
    }

accounts = ["Miccighel_"]
serializer = Serializer("data")
users = serializer.read_json("all_users.json")
edges = []
for account in accounts:
    serializer = Serializer(f'data/{account}')
    account_json = serializer.read_json(f"{account}_profile.json")
    account_id = account_json["id"]
    for user in users:
        if user["id"] is not account_id:
            friendship = get_friendship(account_id, user["id"], api)
            edges.append(friendship)
            if friendship["friendship"] != "none":
                print(friendship)
            print(f"Added friendship between {account} and {user['screen_name']}")
    serializer.serialize_json(f'{account}_friendships.json', edges)

Data read from path: data/all_users.json
Data read from path: data/Miccighel_/Miccighel__profile.json
Added friendship between Miccighel_ and Diana_13721
Added friendship between Miccighel_ and LostQue25565910
Added friendship between Miccighel_ and micros2021
Added friendship between Miccighel_ and PopMan90643879
Added friendship between Miccighel_ and colleen_chen_
Added friendship between Miccighel_ and manafiali1
{'source_id': 15750573, 'target_id': 14616323, 'friendship': 'r_l'}
Added friendship between Miccighel_ and Duesynapse
Added friendship between Miccighel_ and MsaiedurR
Added friendship between Miccighel_ and covid19mlia
Added friendship between Miccighel_ and IshratRahmanSa1
Added friendship between Miccighel_ and Dr_Oli_Jones
Added friendship between Miccighel_ and Hamdy_Mubarak
Added friendship between Miccighel_ and aleksbobic92
Added friendship between Miccighel_ and jakusg
Added friendship between Miccighel_ and jeanburgess
Added friendship between Miccighel_ and Dis

Added friendship between Miccighel_ and datafuturology
Added friendship between Miccighel_ and trevi
Added friendship between Miccighel_ and maik_froebe
Added friendship between Miccighel_ and __alucic
Added friendship between Miccighel_ and tuantx7110
Added friendship between Miccighel_ and salias
Added friendship between Miccighel_ and shahrzad_naseri
Added friendship between Miccighel_ and deb_mazu
Added friendship between Miccighel_ and SIGIReCom
Added friendship between Miccighel_ and FabianoDeiss
Added friendship between Miccighel_ and diazf_acm
Added friendship between Miccighel_ and FattaneZ
Added friendship between Miccighel_ and d_anubrata
Added friendship between Miccighel_ and mdekstrand
Added friendship between Miccighel_ and iFromm
Added friendship between Miccighel_ and vitordouzi
Added friendship between Miccighel_ and harlleyaugusto
Added friendship between Miccighel_ and tab_larasa
Added friendship between Miccighel_ and pchandarr
Added friendship between Miccighel_ a

Rate limit reached. Sleeping for: 847


Added friendship between Miccighel_ and earissola
Added friendship between Miccighel_ and JToscane
Added friendship between Miccighel_ and OkwenAkah
Added friendship between Miccighel_ and asia_biega
Added friendship between Miccighel_ and miriam_fs
Added friendship between Miccighel_ and UNEDNLP
Added friendship between Miccighel_ and MIT_CSAIL
Added friendship between Miccighel_ and kallumadi
Added friendship between Miccighel_ and panglos00047743
Added friendship between Miccighel_ and HarryButcher8
Added friendship between Miccighel_ and Hu_Hsu
Added friendship between Miccighel_ and TAMANNA80943294
Added friendship between Miccighel_ and gsingers
Added friendship between Miccighel_ and dorrreen
Added friendship between Miccighel_ and phillips_bills
Added friendship between Miccighel_ and CheiaJulio
Added friendship between Miccighel_ and KieselJohannes
Added friendship between Miccighel_ and rzhangpku
Added friendship between Miccighel_ and JamesMariolle
Added friendship between M

Added friendship between Miccighel_ and _nhos_
Added friendship between Miccighel_ and hvdthong
Added friendship between Miccighel_ and foclasa2017
Added friendship between Miccighel_ and maliannejadi
Added friendship between Miccighel_ and YashMosh
Added friendship between Miccighel_ and alvarory_
Added friendship between Miccighel_ and socinformatics
Added friendship between Miccighel_ and shashank27392
Added friendship between Miccighel_ and ICTIR17
Added friendship between Miccighel_ and essir2017
Added friendship between Miccighel_ and GunturuManohar
Added friendship between Miccighel_ and irnewsjapan
Added friendship between Miccighel_ and CAIRWorkshop
Added friendship between Miccighel_ and IAI_group
Added friendship between Miccighel_ and bernn
Added friendship between Miccighel_ and HaythamFayek
Added friendship between Miccighel_ and femoraesg
Added friendship between Miccighel_ and kferris01
Added friendship between Miccighel_ and A_D_Delgado
Added friendship between Miccigh

...

Rate limit reached. Sleeping for: 851


Added friendship between Miccighel_ and grazia_borri
Added friendship between Miccighel_ and CitImmCanada
Added friendship between Miccighel_ and Caradelevingne
Added friendship between Miccighel_ and CarasThighGap
Added friendship between Miccighel_ and luke_brooks
Added friendship between Miccighel_ and janoskians
Added friendship between Miccighel_ and EB_amaramavera
Added friendship between Miccighel_ and AceYOLO8
Added friendship between Miccighel_ and JJwright89
Added friendship between Miccighel_ and Miikluka
Added friendship between Miccighel_ and DrDavisVang
Added friendship between Miccighel_ and Iluvhorsepower
Added friendship between Miccighel_ and Joinponrs
Added friendship between Miccighel_ and jackjordan478
Added friendship between Miccighel_ and GrowingUpADHD
Added friendship between Miccighel_ and MiamisModels
Added friendship between Miccighel_ and JimmyVannawong2
Added friendship between Miccighel_ and jdeschai1
Added friendship between Miccighel_ and Khanya61447802

Added friendship between Miccighel_ and peterpaws
Added friendship between Miccighel_ and wesselkraaij
Added friendship between Miccighel_ and tonygrr
Added friendship between Miccighel_ and donadeb
Added friendship between Miccighel_ and UNED
Added friendship between Miccighel_ and bing
Added friendship between Miccighel_ and marcotravaglio
Added friendship between Miccighel_ and ericschmidt
Added friendship between Miccighel_ and SabinaGuzzanti
Added friendship between Miccighel_ and PrestaMarco
Added friendship between Miccighel_ and williamhersh
Added friendship between Miccighel_ and yudapearl
Added friendship between Miccighel_ and GuillaumeSalha
Added friendship between Miccighel_ and m_zecchin
Added friendship between Miccighel_ and alevalsecchi
Added friendship between Miccighel_ and effecube
Added friendship between Miccighel_ and Riccardi_FVG
Added friendship between Miccighel_ and fissore_f
Added friendship between Miccighel_ and SergioBini68


Rate limit reached. Sleeping for: 850


Added friendship between Miccighel_ and AFlaccilla
Added friendship between Miccighel_ and CiriSince1978
Added friendship between Miccighel_ and giuzan
Added friendship between Miccighel_ and vuotodaria_
Added friendship between Miccighel_ and indiasardana
Added friendship between Miccighel_ and wilfredbulus
Added friendship between Miccighel_ and ELYgonzalez0822
Added friendship between Miccighel_ and nick_hoffman8
Added friendship between Miccighel_ and Mz_DesiPie
Added friendship between Miccighel_ and DonLs71
Added friendship between Miccighel_ and STetelo
Added friendship between Miccighel_ and vossie23
Added friendship between Miccighel_ and f_altruist
Added friendship between Miccighel_ and BSakhakhini
Added friendship between Miccighel_ and zubair_1811
Added friendship between Miccighel_ and Fnalhayan
Added friendship between Miccighel_ and Julian_Risch
Added friendship between Miccighel_ and HPCLab
Added friendship between Miccighel_ and 9meo
Added friendship between Miccighel

In [11]:
####################CON ALL_USER E ALL_FRIENDSHIP######################
directory = "data"
G = nx.Graph(membri=["Lorenzo Bellina" "Francesco Bombassei De Bona", "Andrea Cantarutti", "Gabriele Dominici"])
s = Serializer(f"{directory}")
all_nodes = s.read_json(f"all_users.json")
print(len(list(all_nodes)))
for profile in all_nodes:
    G.add_node(profile["id"], follower_ing = 0, following_ing = 0, **profile)
    print(f'Aggiungo il nodo {profile["name"]}')
all_edges = s.read_json(f"all_friendships.json")
for friendship in all_edges:
    if(friendship["friendship"] != "none"):
        G.add_edge(friendship["source_id"], friendship["target_id"], value = friendship["friendship"])
        print(f'Aggiungo l arco tra {friendship["source_id"]} e {friendship["target_id"]}')

for edge in list(G.edges):
    if G.edges[edge[0]][edge[1]]["value"] == "bi":
        G.nodes[edge[0]]["follower_ing"] += 1
        G.nodes[edge[0]]["following_ing"] += 1
        G.nodes[edge[1]]["follower_ing"] += 1
        G.nodes[edge[1]]["following_ing"] += 1
    elif G.edges[edge[0]][edge[1]]["value"] == "r_l":
        G.nodes[edge[0]]["follower_ing"] += 1
        G.nodes[edge[1]]["following_ing"] += 1
    elif G.edges[edge[0]][edge[1]]["value"] == "l_r":
        G.nodes[edge[1]]["follower_ing"] += 1
        G.nodes[edge[0]]["following_ing"] += 1
            
nx.write_gpickle(G, "graph/graph_networkx.pkl")

Data read from path: data/all_users.json
3130
Aggiungo il nodo 🅳🅸🅰🅽🅰 🅼🅰🆁🆃🅸🅽🅴🆉 👅
Aggiungo il nodo Lost Queen
Aggiungo il nodo MICROS @ ECIR2021
Aggiungo il nodo Pop Man
Aggiungo il nodo Colleen Chen
Aggiungo il nodo manafi.law
Aggiungo il nodo ☀ 𝙅𝙤𝙣𝙖𝙨 𝙊𝙥𝙥𝙚𝙣𝙡𝙖𝙚𝙣𝙙𝙚𝙧
Aggiungo il nodo M Saiedur Rahaman
Aggiungo il nodo Covid19 MLIA
Aggiungo il nodo Ishrat Rahman Sami
Aggiungo il nodo ⏣ Prof. Oliver Jones
Aggiungo il nodo Hamdy Mubarak
Aggiungo il nodo Aleks
Aggiungo il nodo Jake Goldenfein
Aggiungo il nodo Jean Burgess
Aggiungo il nodo Disinfo Cloud
Aggiungo il nodo Stefano Cresci
Aggiungo il nodo Minghui
Aggiungo il nodo PeerJ — CompSci
Aggiungo il nodo Dr. Furaha Abwe
Aggiungo il nodo 🅰🅳🅴🅻🅰 🅻🅾🆁🅴🅽🆉🅾 💖
Aggiungo il nodo Lynda Said Lhadj
Aggiungo il nodo Tim Wiley
Aggiungo il nodo Vincenzo Della Mea
Aggiungo il nodo Lauren Kate Kelly🌹
Aggiungo il nodo Priyanka Naik 改善
Aggiungo il nodo SpongeBobLover
Aggiungo il nodo Dr. Borhan Kazimipour
Aggiungo il nodo embeddia
Aggiungo il nodo JDI_Link
Agg

Aggiungo il nodo NEVROTICA
Aggiungo il nodo Travel to
Aggiungo il nodo Srijarmi Alawiyah
Aggiungo il nodo Kelsey Cannon
Aggiungo il nodo The Realistic world
Aggiungo il nodo Hope for Humanity
Aggiungo il nodo Luca Pighin
Aggiungo il nodo What ICallFashion
Aggiungo il nodo Only Girls Know
Aggiungo il nodo SM
Aggiungo il nodo La Pausa Info
Aggiungo il nodo Dandy Shore
Aggiungo il nodo Francesco Boz
Aggiungo il nodo Maiara Walsh
Aggiungo il nodo SpeakTheRumors str
Aggiungo il nodo Nozioni a caso
Aggiungo il nodo Ronaoke
Aggiungo il nodo Insulti Forbiti
Aggiungo il nodo Роман Пантелеев
Aggiungo il nodo Mariëlle
Aggiungo il nodo Riccardo Zucchetto
Aggiungo il nodo Andrea Pavan
Aggiungo il nodo Mastro Retweet! ด้้้
Aggiungo il nodo Demi Divato
Aggiungo il nodo Casey Neistat
Aggiungo il nodo Pro Loco FVG Eventi
Aggiungo il nodo iir2019
Aggiungo il nodo Henriette Cramer
Aggiungo il nodo AI Memes for Artificially Intelligent Teens
Aggiungo il nodo ECIR 2020
Aggiungo il nodo VICE Life
Aggiungo i

Aggiungo l arco tra 132646210 e 156322219
Aggiungo l arco tra 132646210 e 70900164
Aggiungo l arco tra 132646210 e 3775401076
Aggiungo l arco tra 132646210 e 59217922
Aggiungo l arco tra 132646210 e 4228354103
Aggiungo l arco tra 132646210 e 3305766383
Aggiungo l arco tra 132646210 e 1090911182591086592
Aggiungo l arco tra 132646210 e 563947963
Aggiungo l arco tra 132646210 e 822431809
Aggiungo l arco tra 132646210 e 904533569749508097
Aggiungo l arco tra 132646210 e 9719522
Aggiungo l arco tra 132646210 e 1081241209622822913
Aggiungo l arco tra 132646210 e 224209816
Aggiungo l arco tra 132646210 e 3232640646
Aggiungo l arco tra 132646210 e 1077863989202247681
Aggiungo l arco tra 132646210 e 243082126
Aggiungo l arco tra 132646210 e 2882893927
Aggiungo l arco tra 132646210 e 822405576504344576
Aggiungo l arco tra 132646210 e 1721425735
Aggiungo l arco tra 132646210 e 308799838
Aggiungo l arco tra 132646210 e 827392886
Aggiungo l arco tra 132646210 e 713164296604090368
Aggiungo l arco t

3130
2875


In [ ]:
connected = nx.is_connected(G)
bipartite = nx.is_bipartite(G)

center = nx.center(G)
diameter = nx.diameter(G)
radius = nx.radius(G)

betweenness = nx.betweenness_centrality(G) #normalized=True default
closeness = nx.closeness_centrality(G) #wf=True default Wasserman and Faust improved - incremental closeness??
degree = nx.degree_centrality(G)
in_degree = nx.in_degree_centrality(G)
out_degree = nx.out_degree_centrality(G)
page_rank = nx.pagerank(G) #parametri extra
hits = nx.hits(G) #parametri extra

min_edge_cover = nx.min_edge_cover(G) #extra

omega = nx.omega(G) #extra
sigma = nx.sigma(G) #extra